<a href="https://colab.research.google.com/github/winterForestStump/thesis/blob/main/retrieval/Retrievals_approaches_evaluation_phi3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 MB 9.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 6.4 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.76-cp310-cp310-linux_x86_64.whl size=80963167 sha256=ca77184b375c2dea7728da539964a7d410aee34a53c654f626192a66fe8c9c75
  Stored in directory: /root/.cache/pip/wheels/a0/e5/04/a5fa9e60033548f205f0db5f6ab6f59cd27bd0da7f9c51cfe7
Successfully built llama-cpp-python


In [2]:
!huggingface-cli download microsoft/Phi-3-mini-4k-instruct-gguf Phi-3-mini-4k-instruct-fp16.gguf --local-dir ./models --local-dir-use-symlinks False

/usr/local/lib/python3.10/dist-packages/huggingface_hub/commands/download.py:132: FutureWarning: Ignoring --local-dir-use-symlinks. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(
Phi-3-mini-4k-instruct-fp16.gguf: 100% 7.64G/7.64G [01:07<00:00, 113MB/s]
Download complete. Moving file to models/Phi-3-mini-4k-instruct-fp16.gguf
models/Phi-3-mini-4k-instruct-fp16.gguf


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install langchain langchain-core langchain-community --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 11.4 MB/s eta 0:00:00


In [5]:
from langchain_community.llms import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain_community.llms import LlamaCpp
from langchain_core.output_parsers import JsonOutputParser
from json import JSONDecodeError
from langchain_core.exceptions import OutputParserException
import pandas as pd
import os

In [6]:
TEMP = 0
N_CTX = 4096
N_GPU_L = -1

llm = LlamaCpp(
    model_path="/content/models/Phi-3-mini-4k-instruct-fp16.gguf",
    temperature=TEMP,
    n_ctx=N_CTX,
    n_gpu_layers = N_GPU_L,
    verbose=True
)

llama_model_loader: loaded meta data with 23 key-value pairs and 195 tensors from /content/models/Phi-3-mini-4k-instruct-fp16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = phi3
llama_model_loader: - kv   1:                               general.name str              = Phi3
llama_model_loader: - kv   2:                        phi3.context_length u32              = 4096
llama_model_loader: - kv   3:                      phi3.embedding_length u32              = 3072
llama_model_loader: - kv   4:                   phi3.feed_forward_length u32              = 8192
llama_model_loader: - kv   5:                           phi3.block_count u32              = 32
llama_model_loader: - kv   6:                  phi3.attention.head_count u32              = 32
llama_model_loader: - kv   7:               phi3.attention.head_count

In [7]:
### Retrieval Grader for approach 1

prompt_1 = PromptTemplate(
    template="""
    <|assistant|> You are a grader assessing relevance of a retrieved document to a user question. If the document contains keywords related to the user question, grade it as relevant.
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
    Provide the binary score as a JSON with a single key 'score' and no premable or explaination. <|end|>
    <|user|> Here is the retrieved document: \n\n {document} \n\n Here is the user question: {question} <|end|>
    <|assistant|>
    """,
    input_variables=["question", "document"],
)

retrieval_grader_1 = prompt_1 | llm | JsonOutputParser()

In [8]:
### Retrieval Grader for approach 2

prompt_2 = PromptTemplate(
    template="""
    <|assistant|> You are a grader assessing relevance of a retrieved document to a user question. If the document contains keywords related to the user question, grade it as relevant.
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
    Provide the binary score as a JSON with a single key 'score' and no premable or explaination. <|end|>
    <|user|> Here is the retrieved document: \n\n {document} \n\n Here is the user question: {question_name} <|end|>
    <|assistant|>
    """,
    input_variables=["question_name", "document"],
)

retrieval_grader_2 = prompt_2 | llm | JsonOutputParser()

In [10]:
folder_path = '/content/drive/MyDrive/Thesis/retrievals/'

dataframes = []
for file_name in os.listdir(folder_path):
    if file_name.endswith('.json'):
        approach_name = file_name
        dataframes.append(approach_name)

In [11]:
def calculate_results(approach_number:int, distance: str):
  results_list = []

  for i in range(len(dataframes)):

    if approach_number == 1:
      approach = pd.read_json(os.path.join(folder_path + dataframes[i]))
      try:
        for j in range(len(approach)):
          question = approach['question'][j]
          doc_txt = approach[distance][j]['page_content']
          try:
            answer = retrieval_grader_1.invoke({"question": question, "document": doc_txt})
            results_list.append(pd.DataFrame({"approach": 1,"distance": distance, "question": question, "answer": [answer], "document": doc_txt}))
          except JSONDecodeError as e:
            print(f"JSONDecodeError occurred for question: {question}. Skipping...")
            continue
          except OutputParserException as e:
            print(f"OutputParserException occurred for question: {question}. Skipping...")
            continue
      except KeyError as e:
        print(f"KeyError occurred: {e}. Skipping approach {approach_number} for dataframe {dataframes[i]}")
        continue


    elif approach_number == 2:
      approach = pd.read_json(os.path.join(folder_path + dataframes[i]))
      try:
        for j in range(len(approach)):
          question = approach['question_name'][j]
          doc_txt = approach[distance][j]['page_content']
          try:
            answer = retrieval_grader_2.invoke({"question_name": question, "document": doc_txt})
            results_list.append(pd.DataFrame({"approach": 2,"distance": distance, "question": question, "answer": [answer], "document": doc_txt}))
          except JSONDecodeError as e:
            print(f"JSONDecodeError occurred for question: {question}. Skipping...")
            continue
          except OutputParserException as e:
            print(f"OutputParserException occurred for question: {question}. Skipping...")
            continue
      except KeyError as e:
        print(f"KeyError occurred: {e}. Skipping approach {approach_number} for dataframe {dataframes[i]}")
        continue

  if results_list:
    results = pd.concat(results_list, ignore_index=True)
    score = 0
    for i in range(len(results)):
      if (results['answer'][i] is not None) and (results['answer'][i]['score'] == 'yes'):
        score += 1
    total_score = score/len(results)
    print(f'Total score for approach {approach_number} and distance function {distance} is {total_score}')
    results.to_json(f'/content/drive/MyDrive/Thesis/evaluation/eval_{approach_number}_{distance}.json')
  else:
    print(f"No results to concatenate for approach {approach_number} and distance {distance}")

In [12]:
calculate_results(1, 'cosine')


llama_print_timings:        load time =     515.52 ms
llama_print_timings:      sample time =       3.72 ms /     7 runs   (    0.53 ms per token,  1879.70 tokens per second)
llama_print_timings: prompt eval time =    4585.43 ms /   786 tokens (    5.83 ms per token,   171.41 tokens per second)
llama_print_timings:        eval time =     273.68 ms /     6 runs   (   45.61 ms per token,    21.92 tokens per second)
llama_print_timings:       total time =    4966.56 ms /   792 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     515.52 ms
llama_print_timings:      sample time =      42.14 ms /    68 runs   (    0.62 ms per token,  1613.55 tokens per second)
llama_print_timings: prompt eval time =    3136.35 ms /   608 tokens (    5.16 ms per token,   193.86 tokens per second)
llama_print_timings:        eval time =    2611.39 ms /    68 runs   (   38.40 ms per token,    26.04 tokens per second)
llama_print_timings:       total time =    5876.41 ms /   676 

KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_COCA COLA CO_cosine_2.json
KeyError occurred: 'cosine'. Skipping approach 1 for dataframe results_approach_COCA COLA CO_ip_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_COCA COLA CO_ip_2.json
KeyError occurred: 'cosine'. Skipping approach 1 for dataframe results_approach_COCA COLA CO_l2_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_COCA COLA CO_l2_2.json


Llama.generate: prefix-match hit

llama_print_timings:        load time =     515.52 ms
llama_print_timings:      sample time =      40.66 ms /    65 runs   (    0.63 ms per token,  1598.70 tokens per second)
llama_print_timings: prompt eval time =    2776.16 ms /   516 tokens (    5.38 ms per token,   185.87 tokens per second)
llama_print_timings:        eval time =    2541.68 ms /    64 runs   (   39.71 ms per token,    25.18 tokens per second)
llama_print_timings:       total time =    5424.15 ms /   580 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     515.52 ms
llama_print_timings:      sample time =      47.64 ms /    80 runs   (    0.60 ms per token,  1679.37 tokens per second)
llama_print_timings: prompt eval time =    3559.55 ms /   655 tokens (    5.43 ms per token,   184.01 tokens per second)
llama_print_timings:        eval time =    3162.33 ms /    79 runs   (   40.03 ms per token,    24.98 tokens per second)
llama_print_timings:       to

KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_AMAZON COM INC_cosine_2.json
KeyError occurred: 'cosine'. Skipping approach 1 for dataframe results_approach_AMAZON COM INC_ip_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_AMAZON COM INC_ip_2.json
KeyError occurred: 'cosine'. Skipping approach 1 for dataframe results_approach_AMAZON COM INC_l2_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_AMAZON COM INC_l2_2.json


Llama.generate: prefix-match hit

llama_print_timings:        load time =     515.52 ms
llama_print_timings:      sample time =      41.13 ms /    70 runs   (    0.59 ms per token,  1702.04 tokens per second)
llama_print_timings: prompt eval time =    2976.92 ms /   551 tokens (    5.40 ms per token,   185.09 tokens per second)
llama_print_timings:        eval time =    2736.69 ms /    69 runs   (   39.66 ms per token,    25.21 tokens per second)
llama_print_timings:       total time =    5823.78 ms /   620 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     515.52 ms
llama_print_timings:      sample time =      45.22 ms /    62 runs   (    0.73 ms per token,  1371.20 tokens per second)
llama_print_timings: prompt eval time =    2931.07 ms /   547 tokens (    5.36 ms per token,   186.62 tokens per second)
llama_print_timings:        eval time =    2442.94 ms /    61 runs   (   40.05 ms per token,    24.97 tokens per second)
llama_print_timings:       to

...

KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_MICROSOFT CORP_cosine_2.json
KeyError occurred: 'cosine'. Skipping approach 1 for dataframe results_approach_MICROSOFT CORP_ip_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_MICROSOFT CORP_ip_2.json
KeyError occurred: 'cosine'. Skipping approach 1 for dataframe results_approach_MICROSOFT CORP_l2_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_MICROSOFT CORP_l2_2.json


Llama.generate: prefix-match hit

llama_print_timings:        load time =     515.52 ms
llama_print_timings:      sample time =      57.17 ms /    98 runs   (    0.58 ms per token,  1714.25 tokens per second)
llama_print_timings: prompt eval time =    2607.90 ms /   486 tokens (    5.37 ms per token,   186.36 tokens per second)
llama_print_timings:        eval time =    3826.57 ms /    97 runs   (   39.45 ms per token,    25.35 tokens per second)
llama_print_timings:       total time =    6590.66 ms /   583 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     515.52 ms
llama_print_timings:      sample time =       3.38 ms /     7 runs   (    0.48 ms per token,  2071.62 tokens per second)
llama_print_timings: prompt eval time =    2937.67 ms /   549 tokens (    5.35 ms per token,   186.88 tokens per second)
llama_print_timings:        eval time =     239.67 ms /     6 runs   (   39.94 ms per token,    25.03 tokens per second)
llama_print_timings:       to

KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_Ulta Beauty, Inc._cosine_2.json
KeyError occurred: 'cosine'. Skipping approach 1 for dataframe results_approach_Ulta Beauty, Inc._ip_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_Ulta Beauty, Inc._ip_2.json
KeyError occurred: 'cosine'. Skipping approach 1 for dataframe results_approach_Ulta Beauty, Inc._l2_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_Ulta Beauty, Inc._l2_2.json


Llama.generate: prefix-match hit

llama_print_timings:        load time =     515.52 ms
llama_print_timings:      sample time =       3.75 ms /     7 runs   (    0.54 ms per token,  1868.66 tokens per second)
llama_print_timings: prompt eval time =    2353.15 ms /   435 tokens (    5.41 ms per token,   184.86 tokens per second)
llama_print_timings:        eval time =     233.37 ms /     6 runs   (   38.90 ms per token,    25.71 tokens per second)
llama_print_timings:       total time =    2600.27 ms /   441 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     515.52 ms
llama_print_timings:      sample time =       6.28 ms /    12 runs   (    0.52 ms per token,  1912.05 tokens per second)
llama_print_timings: prompt eval time =    2881.73 ms /   544 tokens (    5.30 ms per token,   188.78 tokens per second)
llama_print_timings:        eval time =     434.79 ms /    11 runs   (   39.53 ms per token,    25.30 tokens per second)
llama_print_timings:       to

KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_AES CORP_cosine_2.json
KeyError occurred: 'cosine'. Skipping approach 1 for dataframe results_approach_AES CORP_ip_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_AES CORP_ip_2.json
KeyError occurred: 'cosine'. Skipping approach 1 for dataframe results_approach_AES CORP_l2_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_AES CORP_l2_2.json
Total score for approach 1 and distance function cosine is 0.5715990453460621


In [ ]:
calculate_results(1, 'ip')

In [ ]:
calculate_results(1, 'l2')

In [ ]:
calculate_results(2, 'cosine')

In [ ]:
calculate_results(2, 'ip')

In [ ]:
calculate_results(2, 'l2')